In [1]:
%config Completer.use_jedi = False
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pprint
import datetime
from datetime import datetime, timedelta
import torch
from pyarrow import csv
import os.path
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import gc
from tqdm import tqdm
import scipy.stats as stats
from sklearn.preprocessing import MinMaxScaler
os.environ['KMP_DUPLICATE_LIB_OK']='True'

from IPython.core.display import display, HTML
display(HTML("<style>.container {width:95% !important;}</style>")) #셀 사이즈 조절

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

#matplotlib 한글
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"ㅂ
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

C:\Users\USER\AppData\Local\Temp\ipykernel_11128\3335098552.py:22: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
df = pd.read_csv('C:/Users/USER/etri_csv/논문/논문자료_v1.csv')

C:\Users\USER\AppData\Local\Temp\ipykernel_11128\428708869.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/USER/etri_csv/논문/논문자료_v1.csv')


In [3]:
df['작업시작시간'] =pd.to_datetime(df['작업시작시간'])

In [142]:
df.set_index('작업시작시간', inplace=True)

In [143]:
df.index =pd.to_datetime(df.index)

In [5]:
df.interpolate('time', inplace=True)

In [6]:
df.fillna(method='bfill', inplace=True)

사용할 태그  

In [7]:
tags_Power = [
 전력 관련 태그
 ]

In [8]:
tags_front=[
    전건조 관련 태그
]

In [9]:
tags_back=[
    후건조 관련 태그
]

In [10]:
tags_QCS=[
    나머지 건조 관련 태그
]

In [11]:
tags = tags_Power+tags_QCS+tags_front+tags_back

15분 단축

In [12]:
df_MES = df.copy()

In [13]:
df_event=df_MES[df_MES['지절']==1]#백업

In [14]:
df_MES.drop(df_event.index, inplace=True)

In [15]:
df_MES_15T= df_MES[tags].resample('15T').mean()

In [ ]:
df_MES_15T.isnull().sum()

In [17]:
df_MES_15T.dropna(inplace=True)

In [18]:
df_MES_15T[['지절','지종','평량']] = df_MES[['지절','지종','평량']].resample('15T').first()

In [19]:
tag = ['지절','지종','평량']+tags

In [20]:
df_MES_15T = pd.concat([df_MES_15T, df_event[tag]]) #15분 줄이고 이벤트 합침

데이터를 지종교체시간 기준 앞위 1시간으로 구성

In [21]:
df_MES_15T.reset_index(inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_10080\2489196510.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_MES_15T.reset_index(inplace=True)


In [22]:
df_MES1 = pd.read_csv('C:/Users/USER/etri_csv/1/0_result/DataSet_v6.csv')

In [ ]:
df_MES1['작업총시간'].describe()

In [24]:
df_MES1['작업시작시간']=pd.to_datetime(df_MES1['작업시작시간'])
df_MES1['작업종료시간']=pd.to_datetime(df_MES1['작업종료시간'])

In [25]:
temp=df_MES1.loc[df_MES1['지절']==1,['작업시작시간', '작업종료시간']]

In [26]:
temp['작업시작 전'] = temp['작업시작시간']+ timedelta(minutes=-60)

In [27]:
temp.reset_index(drop=True, inplace=True)

In [28]:
df_MES_temp=pd.DataFrame()

for i in tqdm(range(len(temp))):
    df_MES_temp=pd.concat([df_MES_temp, df_MES_15T[(temp.loc[i,'작업시작 전']<=df_MES_15T['작업시작시간']) & (temp.loc[i,'작업종료시간']>=df_MES_15T['작업시작시간'])]])

100%|█████████████████████████████████████████████████████████████████████████████████| 89/89 [00:00<00:00, 484.41it/s]


In [29]:
df_MES_temp=df_MES_temp.sort_values('작업시작시간')

In [30]:
df_MES_15T=df_MES_temp.drop_duplicates(['작업시작시간'], keep='first')

In [31]:
df_MES_temp=pd.DataFrame()

for i in tqdm(range(len(temp))):
    df_MES_temp=pd.concat([df_MES_temp, df_MES_15T[(temp.loc[i,'작업시작 전']<=df_MES_15T['작업시작시간']) & (temp.loc[i,'작업종료시간']>=df_MES_15T['작업시작시간'])]])

df_MES_temp=df_MES_temp.sort_values('작업시작시간')

df_MES_15T=df_MES_temp.drop_duplicates(['작업시작시간'], keep='first')

100%|█████████████████████████████████████████████████████████████████████████████████| 89/89 [00:00<00:00, 608.38it/s]


지절 발생이 가장 많은 3월 데이터 사용

In [32]:
df_3M_15T =df_MES_15T[df_MES_15T['작업시작시간'].dt.month==3]

#### 상관계수

레이블 인코딩

In [33]:
df_3M_15T.reset_index(drop=True, inplace=True)

In [34]:
condition_1 = ((df_3M_15T['지종']=='ㄱㄱ종이')&(df_3M_15T['평량']=='1'))
condition_2 = ((df_3M_15T['지종']=='ㄴㄴ종이')&(df_3M_15T['평량']=='2'))
condition_3 = ((df_3M_15T['지종']=='ㄷㄷ종이')&(df_3M_15T['평량']=='3'))
condition_4 = ((df_3M_15T['지종']=='ㄹㄹ종이')&(df_3M_15T['평량']=='4'))
condition_5 = ((df_3M_15T['지종']=='ㅁㅁ종이')&(df_3M_15T['평량']=='5'))
condition_6 = ((df_3M_15T['지종']=='ㅂㅂ종이')&(df_3M_15T['평량']=='6'))
condition_7 = ((df_3M_15T['지종']=='ㅅㅅ종이')&(df_3M_15T['평량']=='7'))
condition_8 = ((df_3M_15T['지종']=='ㅇㅇ종이')&(df_3M_15T['평량']=='8'))
condition_9 = ((df_3M_15T['지종']=='ㅈㅈ종이')&(df_3M_15T['평량']=='9'))

In [35]:
condition_legend=['ㄱㄱ종이(1)', 
                  'ㄴㄴ종이(2)',
                  'ㄷㄷ종이(3)',
                  'ㄹㄹ종이(4)',
                  'ㅁㅁ종이(5)', 
                  'ㅂㅂ종이(6)',
                  'ㅅㅅ종이(7)',
                  'ㅇㅇ종이(8)', 
                  'ㅈㅈ종이(9)',
                  ]

In [36]:
conditions=condition_1 | condition_2 | condition_3 | condition_4 | condition_5 | condition_6 | condition_7 | condition_8 | condition_9

In [37]:
tags = ['작업시작시간','지절','지종','평량']+tags_Power+tags_QCS+tags_front+tags_back

In [38]:
df_3M_15T_temp = df_3M_15T[conditions][tags]

In [39]:
for index,condition in enumerate([condition_1,condition_2,condition_3,condition_4,condition_5,condition_6,condition_7,condition_8,condition_9]):
    df_3M_15T_temp.loc[condition,['지종']]=index

In [40]:
list(df_3M_15T_temp['지종'].unique())

[8, 4, 5, 3, 6, 1, 2, 0, 7]

스피어만 상관계수

In [ ]:
#tags_QCS+tags_dry+tags_steam
y = df_3M_15T_temp.loc[:, '지절'].values
temp=pd.DataFrame(columns=['Corr', 'P-val'])

tags = tags_QCS+tags_front+tags_back+['지종','평량']

for item in tags:
    X = df_3M_15T_temp.loc[:, item].values
    if ((stats.spearmanr(X,y)[1]<0.05) and (abs(stats.spearmanr(X,y)[0])>0.3)):
        temp.loc[item] = [stats.spearmanr(X, y)[0], stats.spearmanr(X, y)[1]]


temp['Corr_abs'] = abs(temp['Corr'])
target1 = list(temp.index)

temp.sort_values(by = 'Corr_abs', ascending=False)

In [ ]:
#tags_Power
y = df_3M_15T_temp.loc[:, '지절'].values
temp=pd.DataFrame(columns=['Corr', 'P-val'])

tags = tags_Power

for item in tags:
    X = df_3M_15T_temp.loc[:, item].values
    if ((stats.spearmanr(X,y)[1]<0.05)  and (abs(stats.spearmanr(X,y)[0])>0.5)):
        temp.loc[item] = [stats.spearmanr(X, y)[0], stats.spearmanr(X, y)[1]]


temp['Corr_abs'] = abs(temp['Corr'])
target2 = list(temp.index)

temp.sort_values(by = 'Corr_abs', ascending=False)

svm(지종 vs 지절+DCS/QCS+Power)

In [44]:
df.reset_index(inplace=True)

In [81]:
condition_1 = ((df['지종']=='ㄱㄱ종이') & (df['평량']==1))
condition_2 = ((df['지종']=='ㄴㄴ종이') & (df['평량']==2))
condition_3 = ((df['지종']=='ㄷㄷ종이') & (df['평량']==3))
condition_4 = ((df['지종']=='ㄹㄹ종이') & (df['평량']==4)) 
condition_5 = ((df['지종']=='ㅁㅁ종이') & (df['평량']==5))
condition_6 = ((df['지종']=='ㅂㅂ종이') & (df['평량']==6))
condition_7 = ((df['지종']=='ㅅㅅ종이') & (df['평량']==7))
condition_8 = ((df['지종']=='ㅇㅇ종이') & (df['평량']==8))
condition_9 = ((df['지종']=='ㅈㅈ종이') & (df['평량']==9))
condition_10 = ((df['지종']=='ㅊㅊ종이') & (df['평량']==10))
condition_11 = ((df['지종']=='ㅋㅋ종이') & (df['평량']==11))
condition_12 = ((df['지종']=='ㅌㅌ종이') & (df['평량']==12))
condition_13 = ((df['지종']=='ㅍㅍ종이') & (df['평량']==13))
condition_14 = ((df['지종']=='ㅎㅎ종이') & (df['평량']==14))
condition_15 = ((df['지종']=='AA종이') & (df['평량']==15))
condition_16 = ((df['지종']=='BB종이') & (df['평량']==16))
condition_17 = ((df['지종']=='CC종이') & (df['평량']==17))                   
condition_18 = ((df['지종']=='DD종이') & (df['평량']==18))
condition_19 = ((df['지종']=='EE종이') & (df['평량']==19))     
condition_20 = ((df['지종']=='FF종이') & (df['평량']==20))    

In [83]:
condition_legend2=['ㄱㄱ종이(1)',
                   'ㄴㄴ종이(2)',
                   'ㄷㄷ종이(3)',
                   'ㄹㄹ종이(4)',
                   'ㅁㅁ종이(5)', 
                   'ㅂㅂ종이(6)',
                   'ㅅㅅ종이(7)',
                   'ㅇㅇ종이(8)', 
                   'ㅈㅈ종이(9)',
                   'ㅊㅊ종이(10)',
                   'ㅋㅋ종이(11)',
                   'ㅌㅌ종이(12)',
                   'ㅍㅍ종이(13)',
                   'ㅎㅎ종이(14)',
                   'AA종이(15)',
                   'BB종이(16)',
                   'CC종이(17)',
                   'DD종이(18)',
                   'EE종이(19)',
                   'FF종이(20)'
]

In [84]:
conditions=condition_1 | condition_2 | condition_3 | condition_4 | condition_5 | condition_6 | condition_7 | condition_8 | condition_9 | condition_10 | condition_11 | condition_12 | condition_13 | condition_14 | condition_15 | condition_16 | condition_17 | condition_18 | condition_19 | condition_20

In [85]:
#기존데이터
tags = ['지절','평량','지종']+tags_Power+tags_QCS+tags_front+tags_back

df_temp = df[conditions][tags]

In [86]:
for index,condition in enumerate([condition_1, condition_2, condition_3, condition_4, condition_5, condition_6, condition_7, condition_8, condition_9, condition_10, condition_11, condition_12, condition_13, condition_14, condition_15, condition_16, condition_17, condition_18, condition_19, condition_20]):
    df_temp.loc[condition,['지종']]=index

In [87]:
#svm
target = (target1+target2)+['지절','평량']

df_target = df[conditions][target]

In [88]:
for index,condition in enumerate([condition_1, condition_2, condition_3, condition_4, condition_5, condition_6, condition_7, condition_8, condition_9, condition_10, condition_11, condition_12, condition_13, condition_14, condition_15, condition_16, condition_17, condition_18, condition_19, condition_20]):
    df_target.loc[condition,['지종']]=index

In [128]:
from sklearn.model_selection import train_test_split

seed=1004
sample_ratio=0.1

X_df = df_temp[tags].sample(frac=sample_ratio,random_state=seed)
y_df = df_temp['지종'].sample(frac=sample_ratio,random_state=seed).astype('int')

X=df_target[target].sample(frac=sample_ratio,random_state=seed)
y=df_target['지종'].sample(frac=sample_ratio,random_state=seed).astype('int')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 123)

In [129]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train) # X scaler 생성
X_train = scaler.transform(X_train)   # 스케일링 적용

svm모델 생성

In [135]:
from sklearn import svm

clf= svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)

SVC()

정확도

In [136]:
from sklearn.metrics import accuracy_score

X_test = scaler.transform(X_test)
ddd = pd.DataFrame(X_test,columns=target)

z = clf.predict(X_test)
accuracy_score(y_test, z)

0.9760847283908438

In [ ]:
from sklearn.metrics import confusion_matrix

LABELS =condition_legend

plt.rc('font',)        # 기본 폰트 크기
plt.rc('axes', labelsize=20)   # x,y축 label 폰트 크기
plt.rc('xtick', labelsize=20)  # x축 눈금 폰트 크기 
plt.rc('ytick', labelsize=20)  # y축 눈금 폰트 크기
plt.rc('legend', fontsize=20)  # 범례 폰트 크기
plt.rc('figure', titlesize=50) # figure title 폰트 크기
plt.rcParams['font.family'] = 'Malgun Gothic'

plt.figure(figsize=(15,9))
sns.heatmap(confusion_matrix(y_test, y_pred), xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt='d')
plt.title('Confusion Matrix', size=30)
plt.xlabel('Predicted Class'); plt.ylabel('True Class')



plt.show()

T-SNE

In [59]:
import matplotlib
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')
matplotlib.rcParams['axes.unicode_minus'] = False   
#그래프에서 마이너스 기호가 표시되도록 하는 설정입니다. 

t-SNE 2D

In [60]:
from sklearn.manifold import TSNE

# class target 정보 제외

# origin
tsne_np_o = TSNE(n_components=2).fit_transform(X_df)
tsne_df_o = pd.DataFrame(tsne_np_o, columns = ['component 0', 'component 1'])

#svm
tsne_np_s = TSNE(n_components=2, perplexity=50).fit_transform(X_test)
tsne_df_s = pd.DataFrame(tsne_np_s, columns = ['component 0', 'component 1'])

In [61]:
import matplotlib.pyplot as plt

# class target 정보 불러오기 
tsne_df_o['target'] = y_df.values
tsne_df_s['target'] = y_pred

# 기존
tsne_o_0 = tsne_df_o[tsne_df_o['target'] == 0]
tsne_o_1 = tsne_df_o[tsne_df_o['target'] == 1]
tsne_o_2 = tsne_df_o[tsne_df_o['target'] == 2]
tsne_o_3 = tsne_df_o[tsne_df_o['target'] == 3]
tsne_o_4 = tsne_df_o[tsne_df_o['target'] == 4]
tsne_o_5 = tsne_df_o[tsne_df_o['target'] == 5]
tsne_o_6 = tsne_df_o[tsne_df_o['target'] == 6]
tsne_o_7 = tsne_df_o[tsne_df_o['target'] == 7]
tsne_o_8 = tsne_df_o[tsne_df_o['target'] == 8]
tsne_o_9 = tsne_df_o[tsne_df_o['target'] == 9]
tsne_o_10 = tsne_df_o[tsne_df_o['target'] == 10]
tsne_o_11 = tsne_df_o[tsne_df_o['target'] == 11]
tsne_o_12 = tsne_df_o[tsne_df_o['target'] == 12]
tsne_o_13 = tsne_df_o[tsne_df_o['target'] == 13]
tsne_o_14 = tsne_df_o[tsne_df_o['target'] == 14]
tsne_o_15 = tsne_df_o[tsne_df_o['target'] == 15]
tsne_o_17 = tsne_df_o[tsne_df_o['target'] == 17]

#SVM
tsne_s_0 = tsne_df_s[tsne_df_s['target'] == 0]
tsne_s_1 = tsne_df_s[tsne_df_s['target'] == 1]
tsne_s_2 = tsne_df_s[tsne_df_s['target'] == 2]
tsne_s_3 = tsne_df_s[tsne_df_s['target'] == 3]
tsne_s_4 = tsne_df_s[tsne_df_s['target'] == 4]
tsne_s_5 = tsne_df_s[tsne_df_s['target'] == 5]
tsne_s_6 = tsne_df_s[tsne_df_s['target'] == 6]
tsne_s_7 = tsne_df_s[tsne_df_s['target'] == 7]
tsne_s_8 = tsne_df_s[tsne_df_s['target'] == 8]
tsne_s_9 = tsne_df_s[tsne_df_s['target'] == 9]
tsne_s_10 = tsne_df_s[tsne_df_s['target'] == 10]
tsne_s_11 = tsne_df_s[tsne_df_s['target'] == 11]
tsne_s_12 = tsne_df_s[tsne_df_s['target'] == 12]
tsne_s_13 = tsne_df_s[tsne_df_s['target'] == 13]
tsne_s_14 = tsne_df_s[tsne_df_s['target'] == 14]
tsne_s_15 = tsne_df_s[tsne_df_s['target'] == 15]
tsne_s_17 = tsne_df_s[tsne_df_s['target'] == 17]


plt.rc('axes', labelsize=20)   # x,y축 label 폰트 크기
plt.rc('xtick', labelsize=20)  # x축 눈금 폰트 크기 
plt.rc('ytick', labelsize=20)  # y축 눈금 폰트 크기
plt.rc('legend', fontsize=12)  # 범례 폰트 크기
plt.rc('figure', titlesize=50) # figure title 폰트 크기

# target 별 시각화
plt.figure(figsize = (20,6), constrained_layout = True)

plt.subplot(121)
plt.scatter(tsne_o_0['component 0'], tsne_o_0['component 1'], color = 'darkred', label = '0', s=50)
plt.scatter(tsne_o_1['component 0'], tsne_o_1['component 1'], color = 'rosybrown', label = '1', s=50)
plt.scatter(tsne_o_2['component 0'], tsne_o_2['component 1'], color = 'lightcoral', label = '2', s=50)
plt.scatter(tsne_o_3['component 0'], tsne_o_3['component 1'], color = 'orange', label = '3', s=50)
plt.scatter(tsne_o_4['component 0'], tsne_o_4['component 1'], color = 'peru', label = '4', s=50)
plt.scatter(tsne_o_5['component 0'], tsne_o_5['component 1'], color = 'bisque', label = '5', s=50)
plt.scatter(tsne_o_6['component 0'], tsne_o_6['component 1'], color = 'yellow', label = '6', s=50)
plt.scatter(tsne_o_7['component 0'], tsne_o_7['component 1'], color = 'olive', label = '7', s=50)
plt.scatter(tsne_o_8['component 0'], tsne_o_8['component 1'], color = 'yellowgreen', label = '8', s=50)
plt.scatter(tsne_o_9['component 0'], tsne_o_9['component 1'], color = 'forestgreen', label = '9', s=50)
plt.scatter(tsne_o_10['component 0'], tsne_o_10['component 1'], color = 'aquamarine', label = '10', s=50)
plt.scatter(tsne_o_11['component 0'], tsne_o_11['component 1'], color = 'lightblue', label = '11', s=50)
plt.scatter(tsne_o_12['component 0'], tsne_o_12['component 1'], color = 'deeppink', label = '12', s=50)
plt.scatter(tsne_o_13['component 0'], tsne_o_13['component 1'], color = 'slateblue', label = '13', s=50)
plt.scatter(tsne_o_14['component 0'], tsne_o_14['component 1'], color = 'plum', label = '14', s=50)
plt.scatter(tsne_o_15['component 0'], tsne_o_15['component 1'], color = 'royalblue', label = '15', s=50)
plt.scatter(tsne_o_17['component 0'], tsne_o_17['component 1'], color = 'purple', label = '17', s=50)

plt.title('Original', size=15)
plt.legend(condition_legend, loc=(1.05,0.1))
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.grid()

plt.subplot(122)
plt.scatter(tsne_s_0['component 0'], tsne_s_0['component 1'], color = 'darkred', label = '0', s=50)
plt.scatter(tsne_s_1['component 0'], tsne_s_1['component 1'], color = 'rosybrown', label = '1', s=50)
plt.scatter(tsne_s_2['component 0'], tsne_s_2['component 1'], color = 'lightcoral', label = '2', s=50)
plt.scatter(tsne_s_3['component 0'], tsne_s_3['component 1'], color = 'orange', label = '3', s=50)
plt.scatter(tsne_s_4['component 0'], tsne_s_4['component 1'], color = 'peru', label = '4', s=50)
plt.scatter(tsne_s_5['component 0'], tsne_s_5['component 1'], color = 'bisque', label = '5', s=50)
plt.scatter(tsne_s_6['component 0'], tsne_s_6['component 1'], color = 'yellow', label = '6', s=50)
plt.scatter(tsne_s_7['component 0'], tsne_s_7['component 1'], color = 'olive', label = '7', s=50)
plt.scatter(tsne_s_8['component 0'], tsne_s_8['component 1'], color = 'yellowgreen', label = '8', s=50)
plt.scatter(tsne_s_9['component 0'], tsne_s_9['component 1'], color = 'forestgreen', label = '9', s=50)
plt.scatter(tsne_s_10['component 0'], tsne_s_10['component 1'], color = 'aquamarine', label = '10', s=50)
plt.scatter(tsne_s_11['component 0'], tsne_s_11['component 1'], color = 'lightblue', label = '11', s=50)
plt.scatter(tsne_s_12['component 0'], tsne_s_12['component 1'], color = 'deeppink', label = '12', s=50)
plt.scatter(tsne_s_13['component 0'], tsne_s_13['component 1'], color = 'slateblue', label = '13', s=50)
plt.scatter(tsne_s_14['component 0'], tsne_s_14['component 1'], color = 'plum', label = '14', s=50)
plt.scatter(tsne_s_15['component 0'], tsne_s_15['component 1'], color = 'royalblue', label = '15', s=50)
plt.scatter(tsne_s_17['component 0'], tsne_s_17['component 1'], color = 'purple', label = '17', s=50)

plt.title('SVM result', size=15)
plt.legend(condition_legend, loc=(1.05,0.1))
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.grid()

plt.show()

t-SNE 3D

In [62]:
from sklearn.manifold import TSNE

#original
tsne_np_o = TSNE(n_components = 3).fit_transform(X_df)
tsne_df_o = pd.DataFrame(tsne_np_o, columns = ['component 0', 'component 1', 'component2'])

#original
tsne_np_s = TSNE(n_components = 3, perplexity=50).fit_transform(X_test)
tsne_df_s = pd.DataFrame(tsne_np_s, columns = ['component 0', 'component 1', 'component2'])

In [63]:
import matplotlib.pyplot as plt

# class target 정보 불러오기 
tsne_df_o['target'] = y_df.values
tsne_df_s['target'] = y_pred

# 기존
tsne_o_0 = tsne_df_o[tsne_df_o['target'] == 0]
tsne_o_1 = tsne_df_o[tsne_df_o['target'] == 1]
tsne_o_2 = tsne_df_o[tsne_df_o['target'] == 2]
tsne_o_3 = tsne_df_o[tsne_df_o['target'] == 3]
tsne_o_4 = tsne_df_o[tsne_df_o['target'] == 4]
tsne_o_5 = tsne_df_o[tsne_df_o['target'] == 5]
tsne_o_6 = tsne_df_o[tsne_df_o['target'] == 6]
tsne_o_7 = tsne_df_o[tsne_df_o['target'] == 7]
tsne_o_8 = tsne_df_o[tsne_df_o['target'] == 8]
tsne_o_9 = tsne_df_o[tsne_df_o['target'] == 9]
tsne_o_10 = tsne_df_o[tsne_df_o['target'] == 10]
tsne_o_11 = tsne_df_o[tsne_df_o['target'] == 11]
tsne_o_12 = tsne_df_o[tsne_df_o['target'] == 12]
tsne_o_13 = tsne_df_o[tsne_df_o['target'] == 13]
tsne_o_14 = tsne_df_o[tsne_df_o['target'] == 14]
tsne_o_15 = tsne_df_o[tsne_df_o['target'] == 15]
tsne_o_17 = tsne_df_o[tsne_df_o['target'] == 17]

#SVM
tsne_s_0 = tsne_df_s[tsne_df_s['target'] == 0]
tsne_s_1 = tsne_df_s[tsne_df_s['target'] == 1]
tsne_s_2 = tsne_df_s[tsne_df_s['target'] == 2]
tsne_s_3 = tsne_df_s[tsne_df_s['target'] == 3]
tsne_s_4 = tsne_df_s[tsne_df_s['target'] == 4]
tsne_s_5 = tsne_df_s[tsne_df_s['target'] == 5]
tsne_s_6 = tsne_df_s[tsne_df_s['target'] == 6]
tsne_s_7 = tsne_df_s[tsne_df_s['target'] == 7]
tsne_s_8 = tsne_df_s[tsne_df_s['target'] == 8]
tsne_s_9 = tsne_df_s[tsne_df_s['target'] == 9]
tsne_s_10 = tsne_df_s[tsne_df_s['target'] == 10]
tsne_s_11 = tsne_df_s[tsne_df_s['target'] == 11]
tsne_s_12 = tsne_df_s[tsne_df_s['target'] == 12]
tsne_s_13 = tsne_df_s[tsne_df_s['target'] == 13]
tsne_s_14 = tsne_df_s[tsne_df_s['target'] == 14]
tsne_s_15 = tsne_df_s[tsne_df_s['target'] == 15]
tsne_s_17 = tsne_df_s[tsne_df_s['target'] == 17]

# target 별 시각화
fig = plt.figure(figsize=(17,17))

plt.rc('axes', labelsize=20)   # x,y축 label 폰트 크기
plt.rc('xtick', labelsize=20)  # x축 눈금 폰트 크기 
plt.rc('ytick', labelsize=20)  # y축 눈금 폰트 크기
plt.rc('legend', fontsize=20)  # 범례 폰트 크기
#plt.rc('figure', titlesize=50) # figure title 폰트 크기

ax = fig.add_subplot(121, projection='3d')
ax.scatter(tsne_o_0['component 0'], tsne_o_0['component 1'], tsne_o_0['component2'], color = 'darkred', label = '0', s=25)
ax.scatter(tsne_o_1['component 0'], tsne_o_1['component 1'], tsne_o_1['component2'], color = 'rosybrown', label = '1', s=25)
ax.scatter(tsne_o_2['component 0'], tsne_o_2['component 1'], tsne_o_2['component2'], color = 'lightcoral', label = '2', s=25)
ax.scatter(tsne_o_3['component 0'], tsne_o_3['component 1'], tsne_o_3['component2'], color = 'orange', label = '3', s=25)
ax.scatter(tsne_o_4['component 0'], tsne_o_4['component 1'], tsne_o_4['component2'], color = 'peru', label = '4', s=25)
ax.scatter(tsne_o_5['component 0'], tsne_o_5['component 1'], tsne_o_5['component2'], color = 'bisque', label = '5', s=25)
ax.scatter(tsne_o_6['component 0'], tsne_o_6['component 1'], tsne_o_6['component2'], color = 'yellow', label = '6', s=25)
ax.scatter(tsne_o_7['component 0'], tsne_o_7['component 1'], tsne_o_7['component2'], color = 'olive', label = '7', s=25)
ax.scatter(tsne_o_8['component 0'], tsne_o_8['component 1'], tsne_o_8['component2'], color = 'yellowgreen', label = '8', s=25)
ax.scatter(tsne_o_9['component 0'], tsne_o_9['component 1'], tsne_o_9['component2'], color = 'forestgreen', label = '9', s=25)
ax.scatter(tsne_o_10['component 0'], tsne_o_10['component 1'], tsne_o_10['component2'], color = 'aquamarine', label = '10', s=25)
ax.scatter(tsne_o_11['component 0'], tsne_o_11['component 1'], tsne_o_11['component2'], color = 'lightblue', label = '11', s=25)
ax.scatter(tsne_o_12['component 0'], tsne_o_12['component 1'], tsne_o_12['component2'], color = 'deeppink', label = '12', s=25)
ax.scatter(tsne_o_13['component 0'], tsne_o_13['component 1'], tsne_o_13['component2'], color = 'slateblue', label = '13', s=25)
ax.scatter(tsne_o_14['component 0'], tsne_o_14['component 1'], tsne_o_14['component2'], color = 'plum', label = '14', s=25)
ax.scatter(tsne_o_15['component 0'], tsne_o_15['component 1'], tsne_o_15['component2'], color = 'royalblue', label = '15', s=25)
ax.scatter(tsne_o_17['component 0'], tsne_o_17['component 1'], tsne_o_17['component2'], color = 'purple', label = '17', s=25)

ax.set_xlabel('Component 1', labelpad=17)
ax.set_ylabel('Component 2', labelpad=17)
ax.set_zlabel('Component 3', labelpad=17)
ax.set_title('Original', fontsize=20)
ax.view_init(15,50)
ax.grid()

plt.subplots_adjust(wspace=0.3)

ax = fig.add_subplot(122, projection='3d')
ax.scatter(tsne_s_0['component 0'], tsne_s_0['component 1'], tsne_s_0['component2'], color = 'darkred', label = '0', s=25)
ax.scatter(tsne_s_1['component 0'], tsne_s_1['component 1'], tsne_s_1['component2'], color = 'rosybrown', label = '1', s=25)
ax.scatter(tsne_s_2['component 0'], tsne_s_2['component 1'], tsne_s_2['component2'], color = 'lightcoral', label = '2', s=25)
ax.scatter(tsne_s_3['component 0'], tsne_s_3['component 1'], tsne_s_3['component2'], color = 'orange', label = '3', s=25)
ax.scatter(tsne_s_4['component 0'], tsne_s_4['component 1'], tsne_s_4['component2'], color = 'peru', label = '4', s=25)
ax.scatter(tsne_s_5['component 0'], tsne_s_5['component 1'], tsne_s_5['component2'], color = 'bisque', label = '5', s=25)
ax.scatter(tsne_s_6['component 0'], tsne_s_6['component 1'], tsne_s_6['component2'], color = 'yellow', label = '6', s=25)
ax.scatter(tsne_s_7['component 0'], tsne_s_7['component 1'], tsne_s_7['component2'], color = 'olive', label = '7', s=25)
ax.scatter(tsne_s_8['component 0'], tsne_s_8['component 1'], tsne_s_8['component2'], color = 'yellowgreen', label = '8', s=25)
ax.scatter(tsne_s_9['component 0'], tsne_s_9['component 1'], tsne_s_9['component2'], color = 'forestgreen', label = '9', s=25)
ax.scatter(tsne_s_10['component 0'], tsne_s_10['component 1'], tsne_s_10['component2'], color = 'aquamarine', label = '10', s=25)
ax.scatter(tsne_s_11['component 0'], tsne_s_11['component 1'], tsne_s_11['component2'], color = 'lightblue', label = '11', s=25)
ax.scatter(tsne_s_12['component 0'], tsne_s_12['component 1'], tsne_s_12['component2'], color = 'deeppink', label = '12', s=25)
ax.scatter(tsne_s_13['component 0'], tsne_s_13['component 1'], tsne_s_13['component2'], color = 'slateblue', label = '13', s=25)
ax.scatter(tsne_s_14['component 0'], tsne_s_14['component 1'], tsne_s_14['component2'], color = 'plum', label = '14', s=25)
ax.scatter(tsne_s_15['component 0'], tsne_s_15['component 1'], tsne_s_15['component2'], color = 'royalblue', label = '15', s=25)
ax.scatter(tsne_s_17['component 0'], tsne_s_17['component 1'], tsne_s_17['component2'], color = 'purple', label = '17', s=25)
ax.legend(condition_legend, loc=(1.05,0.2), prop={'size': 15})

ax.set_xlabel('Component 1', labelpad=17)
ax.set_ylabel('Component 2', labelpad=17)
ax.set_zlabel('Component 3', labelpad=17)
ax.set_title('SVM result', fontsize=20)
ax.view_init(15,50)
ax.grid()

plt.show()